In [ ]:
# !uv add loguru pydantic rich tenacity

<img src="./docs/prompt_chaining1.jpg" alt="Prompt Chaining" width="1200"/>
<img src="./docs/prompt_chaining2.jpg" alt="Prompt Chaining" width="1200"/>

In [ ]:
import asyncio
import os

from agents import Runner, RunResult, trace
from build_agents import curriculum_agent, curriculum_checker_agent, lesson_writer_agent
from configs import CurriculumCheckOutput
from rich.console import Console
from rich.markdown import Markdown

# from IPython.display import Markdown, display

console = Console()

# learning_goal: str = input(input_prompt)
learning_goal = "I want to learn Python "

with trace(workflow_name="Deterministic tutor flow"):
    # 1. Generate curriculum
    print(f"INFO | Generating curriculum for goal: {learning_goal}")
    curriculum_result: RunResult = await Runner.run(
        starting_agent=curriculum_agent,
        input=learning_goal,
    )
    console.print(
        Markdown(f"Curriculum generated.\n{curriculum_result.final_output}\n")
    )
    
    # 2. Check the curriculum
    print("INFO | Checking the curriculum quality and goal match...")
    check_result: RunResult = await Runner.run(
        starting_agent=curriculum_checker_agent,
        input=curriculum_result.final_output,
    )
    
    assert isinstance(check_result.final_output, CurriculumCheckOutput)
    if not check_result.final_output.good_quality:
        print("\n")
        print("INFO | Curriculum is low quality. Stopping.")
        exit(0)
    
    if not check_result.final_output.matches_goal:
        print("\n")
        print("INFO | Curriculum doesn't match the learning goal. Stopping.")
        exit(0)
    
    print("\nINFO | Curriculum looks good. Proceeding to generate lessons...")
    
    # 3. Generate lessons
    lessons_result: RunResult = await Runner.run(
        starting_agent=lesson_writer_agent,
        input=curriculum_result.final_output,
    )
    console.print(Markdown(f"\nGenerated Lessons:\n{lessons_result.final_output}"))